# RESNET18-FER2013

Notebook này trình bày cách huấn luyện và đánh giá mô hình **Resnet18** trên tập dữ liệu cảm xúc khuôn mặt **FER2013**.

Nội dung gồm:
1.	Tải và tiền xử lý dữ liệu
2.	Load mô hình resnet18  và thực hiện fintune mô hình với fer2013
3.	Huấn luyện mô hình
4.	Đánh giá: Độ chính xác (Accuracy), Ma trận nhầm lẫn (Confusion Matrix), Thời gian suy luận (Inference Time), Kích thước mô hình (Model Size)

Cấu trúc thư mục dữ liệu phải được tổ chức thành hai thư mục train/ và test/, mỗi thư mục chứa các thư mục con tương ứng với từng nhãn cảm xúc.

## 1. Chuẩn bị và tải dữ liệu

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, random_split
from torchvision.models import ResNet18_Weights


In [27]:
# 📦 Các cấu hình
IMG_SIZE = 224
BATCH_SIZE = 32

train_path = '../../Bai_4/data/train'
test_path = '../../Bai_4/data/test'

train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.RandomAffine(0, shear=15, scale=(0.8,1.2)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor()
])

val_test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor()
])

# 📚 Load tập train ban đầu
full_train_dataset = datasets.ImageFolder(root=train_path, transform=train_transform)

# 🧩 Tách train/val theo tỷ lệ
train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

# Nhưng chú ý: validation cần **không** dùng data augmentation -> chỉnh transform
val_dataset.dataset.transform = val_test_transform

# 📚 Load tập test
test_dataset = datasets.ImageFolder(root=test_path, transform=val_test_transform)

# 🚀 Tạo DataLoader
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Số lượng ảnh trong từng tập
print(f"Số lượng ảnh trong tập huấn luyện: {len(train_dataset)}")
print(f"Số lượng ảnh trong tập validation: {len(val_dataset)}")
print(f"Số lượng ảnh trong tập test: {len(test_dataset)}")



Số lượng ảnh trong tập huấn luyện: 22967
Số lượng ảnh trong tập validation: 5742
Số lượng ảnh trong tập test: 7178


## 2. Khời tạo mô hình resnet18

In [28]:
# Khởi tạo mô hình ResNet18 với trọng số đã được huấn luyện trên ImageNet
model = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

# Sửa lại lớp fully connected cuối cùng để phù hợp với số lớp trong FER2013 (7 lớp)
model.fc = nn.Linear(model.fc.in_features, 7)

# Chuyển mô hình sang GPU (nếu có GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
print(device)


cpu


##  3.Loss function và adam optimize

Chúng ta sẽ sử dụng CrossEntropyLoss cho bài toán phân loại đa lớp và Adam optimizer cho việc tối ưu hóa mô hình.

In [29]:
# Loss function (CrossEntropy cho phân loại đa lớp)
criterion = nn.CrossEntropyLoss()

# Optimizer (Adam optimizer)
optimizer = optim.Adam(model.parameters(), lr=0.001)


## 4. Huấn luyện mô hình

In [30]:
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

NUM_EPOCHS = 1  # hoặc số epoch bạn muốn

for epoch in range(NUM_EPOCHS):
    # ----- Train -----
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)

    train_loss = running_loss / len(train_loader)
    train_acc = correct_train / total_train
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)

    # ----- Validation -----
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct_val += (preds == labels).sum().item()
            total_val += labels.size(0)

    val_loss /= len(val_loader)
    val_acc = correct_val / total_val
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] - "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} - "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")


KeyboardInterrupt: 